#Congo

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("/content/AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    exit()

df_congo = df[df["COUNTRY"].isin(["Congo", "Democratic Republic of the Congo"])].copy()

try:
    df_temp = pd.read_csv("/content/CongoTemp_1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("/content/CongoPrecipitation_1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files not found. Please ensure they are in the correct directory.")
    exit()

climate_min_year = 1984
climate_max_year = 2020
df_congo = df_congo[(df_congo['YY'] >= climate_min_year) & (df_congo['YY'] <= climate_max_year)].copy()

print(f"Shape of df_congo after initial country and YEAR filter: {df_congo.shape}")
print(f"Years: Min={df_congo['YY'].min() if not df_congo.empty else 'N/A'}, Max={df_congo['YY'].max() if not df_congo.empty else 'N/A'}")

if df_congo.empty:
    print("df_congo is empty after filtering.")
    exit()

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

df_congo["Lat_Rounded"] = df_congo["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_congo["Long_Rounded"] = df_congo["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_congo["Month_Name"] = df_congo["MM"].map(month_map)
df_congo["Temperature"] = np.nan
df_congo["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_congo.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_congo.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_congo.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation ---
df_congo = df_congo.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_congo = df_congo[(df_congo["Ex"] > 0) & (df_congo["Pf"] <= df_congo["Ex"])]
df_congo["Pf"] = df_congo["Pf"].astype(int)
df_congo["Ex"] = df_congo["Ex"].astype(int)

if df_congo.empty:
    print("\nError: No valid data points remaining.")
    exit()

temp_mean = df_congo["Temperature"].mean()
temp_std = df_congo["Temperature"].std()
precip_mean = df_congo["Precipitation"].mean()
precip_std = df_congo["Precipitation"].std()

df_congo["Temperature_std"] = (df_congo["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_congo["Precipitation_std"] = (df_congo["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
with pm.Model() as climate_model_congo:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
    beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
    beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
    beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

    logit_p = (
        intercept
        + beta_temp * df_congo["Temperature_std"].values
        + beta_temp2 * df_congo["Temperature_std"].values**2
        + beta_precip * df_congo["Precipitation_std"].values
        + beta_precip2 * df_congo["Precipitation_std"].values**2
    )
    p = pm.Deterministic("p", pm.math.sigmoid(logit_p))
    y_obs = pm.Binomial("y_obs", n=df_congo["Ex"].values, p=p, observed=df_congo["Pf"].values)

    trace_climate_congo = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 5. Baseline Model ---
with pm.Model() as baseline_model_congo:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    p = pm.Deterministic("p", pm.math.sigmoid(intercept))
    y_obs = pm.Binomial("y_obs", n=df_congo["Ex"].values, p=p, observed=df_congo["Pf"].values)

    trace_baseline_congo = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 6. Model Comparison Plot ---
p_climate_congo = trace_climate_congo.posterior["p"].mean(dim=["chain", "draw"]).values
p_baseline_congo = trace_baseline_congo.posterior["p"].mean(dim=["chain", "draw"]).values

plt.figure(figsize=(12, 6))
plt.plot(p_climate_congo, label="Climate Model", color='skyblue')
plt.plot(p_baseline_congo, label="Baseline Model", linestyle='--', color='salmon')
plt.ylabel("Predicted Malaria Prevalence")
plt.xlabel("Observation Index")
plt.legend()
plt.title("Climate vs Baseline Model: Congo/DRC")
plt.grid(True, linestyle=':')
plt.show()

# --- 7. Predicted vs Temperature (fixed Precipitation) ---
intercept_vals = trace_climate_congo.posterior["intercept"].stack(sample=("chain", "draw")).values
beta_temp_vals = trace_climate_congo.posterior["beta_temp"].stack(sample=("chain", "draw")).values
beta_precip_vals = trace_climate_congo.posterior["beta_precip"].stack(sample=("chain", "draw")).values
beta_temp2_vals = trace_climate_congo.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
beta_precip2_vals = trace_climate_congo.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

temp_range_raw = np.linspace(df_congo["Temperature"].min()-5, df_congo["Temperature"].max()+5, 100)
precip_const_raw = df_congo["Precipitation"].mean()
temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

pred_probs_temp = []
for temp_std_val in temp_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_std_val
        + beta_temp2_vals * temp_std_val**2
        + beta_precip_vals * precip_const_std
        + beta_precip2_vals * precip_const_std**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_temp.append(prob)

pred_probs_temp = np.array(pred_probs_temp)
mean_pred_temp = pred_probs_temp.mean(axis=1)
lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Temperature (Precip = {precip_const_raw:.2f} mm/day)")
plt.grid(True, linestyle=':')
plt.show()

# --- 8. Predicted vs Precipitation (fixed Temperature) ---
precip_range_raw = np.linspace(df_congo["Precipitation"].min()-5, df_congo["Precipitation"].max()+5, 100)
temp_const_raw = df_congo["Temperature"].mean()
precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

pred_probs_precip = []
for precip_std_val in precip_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_const_std
        + beta_temp2_vals * temp_const_std**2
        + beta_precip_vals * precip_std_val
        + beta_precip2_vals * precip_std_val**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_precip.append(prob)

pred_probs_precip = np.array(pred_probs_precip)
mean_pred_precip = pred_probs_precip.mean(axis=1)
lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Precipitation (Temp = {temp_const_raw:.2f}°C)")
plt.grid(True, linestyle=':')
plt.show()

# --- 9. Varying Precipitation (5 levels) ---
precip_levels = np.percentile(df_congo["Precipitation"], [10, 30, 50, 70, 90])
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

plt.figure(figsize=(10, 6))
for i, precip_val in enumerate(precip_levels):
    precip_std = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
    preds = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
            beta_precip_vals * precip_std + beta_precip2_vals * precip_std**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Temperature (5 Precip Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.show()

# --- 10. Varying Temperature (5 levels) ---
temp_levels = np.percentile(df_congo["Temperature"], [10, 30, 50, 70, 90])
colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

plt.figure(figsize=(10, 6))
for i, temp_val in enumerate(temp_levels):
    temp_std = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
    preds = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std + beta_temp2_vals * temp_std**2 +
            beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Precipitation (5 Temp Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.show()

# --- 11. Summary ---
print("\n--- Climate Model Summary ---")
print(az.summary(trace_climate_congo, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

print("\n--- Baseline Model Summary ---")
print(az.summary(trace_baseline_congo, var_names=["intercept"]))

In [ ]:
import seaborn as sns
# --- Scatter Plots ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(df_congo["Temperature"], df_congo["PfPR2-10"], alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Temperature vs Malaria Prevalence")

plt.subplot(1, 2, 2)
plt.scatter(df_congo["Precipitation"], df_congo["PfPR2-10"], alpha=0.6, color='green')
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Precipitation vs Malaria Prevalence")
plt.tight_layout()
plt.show()

# --- Monthly Climate and Prevalence Trends ---
monthly_temp = df_congo.groupby("MM")["Temperature"].mean()
monthly_precip = df_congo.groupby("MM")["Precipitation"].mean()
monthly_pfpr = df_congo.groupby("MM")["PfPR2-10"].mean()

plt.figure(figsize=(15, 6))

plt.subplot(2, 3, 1)
monthly_temp.plot(marker='o')
plt.title("Seasonal Temperature Pattern")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")

plt.subplot(2, 3, 2)
monthly_precip.plot(marker='o', color='blue')
plt.title("Seasonal Precipitation Pattern")
plt.xlabel("Month")
plt.ylabel("Precipitation (mm/day)")

plt.subplot(2, 3, 3)
monthly_pfpr.plot(marker='o', color='red')
plt.title("Seasonal Malaria Pattern")
plt.xlabel("Month")
plt.ylabel("PfPR2-10 (%)")

# --- Heatmap of Binned Temp × Precip vs Prevalence ---
df_congo["T_bin"] = pd.cut(df_congo["Temperature"], bins=5, labels=False)
df_congo["P_bin"] = pd.cut(df_congo["Precipitation"], bins=5, labels=False)
heatmap_data = df_congo.groupby(["T_bin", "P_bin"])["PfPR2-10"].mean().unstack()

plt.subplot(2, 3, 6)
sns.heatmap(heatmap_data, cmap="YlOrRd", annot=True, fmt=".1f", cbar_kws={'label': 'PfPR2-10 (%)'})
plt.title("Climate-Malaria Heatmap")
plt.xlabel("Precipitation Bins")
plt.ylabel("Temperature Bins")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("/content/AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    exit()

df_congo = df[df["COUNTRY"].isin(["Congo", "Democratic Republic of the Congo"])].copy()

try:
    df_temp = pd.read_csv("/content/CongoTemp_1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("/content/CongoPrecipitation_1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files not found. Please ensure they are in the correct directory.")
    exit()

climate_min_year = 1984
climate_max_year = 2020
df_congo = df_congo[(df_congo['YY'] >= climate_min_year) & (df_congo['YY'] <= climate_max_year)].copy()

print(f"Shape of df_congo after initial country and YEAR filter: {df_congo.shape}")
print(f"Years: Min={df_congo['YY'].min() if not df_congo.empty else 'N/A'}, Max={df_congo['YY'].max() if not df_congo.empty else 'N/A'}")

if df_congo.empty:
    print("df_congo is empty after filtering.")
    exit()

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

df_congo["Lat_Rounded"] = df_congo["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_congo["Long_Rounded"] = df_congo["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_congo["Month_Name"] = df_congo["MM"].map(month_map)
df_congo["Temperature"] = np.nan
df_congo["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_congo.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_congo.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_congo.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation ---
df_congo = df_congo.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_congo = df_congo[(df_congo["Ex"] > 0) & (df_congo["Pf"] <= df_congo["Ex"])]
df_congo["Pf"] = df_congo["Pf"].astype(int)
df_congo["Ex"] = df_congo["Ex"].astype(int)

if df_congo.empty:
    print("\nError: No valid data points remaining.")
    exit()

temp_mean = df_congo["Temperature"].mean()
temp_std = df_congo["Temperature"].std()
precip_mean = df_congo["Precipitation"].mean()
precip_std = df_congo["Precipitation"].std()

df_congo["Temperature_std"] = (df_congo["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_congo["Precipitation_std"] = (df_congo["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
with pm.Model() as climate_model_congo:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
    beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
    beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
    beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

    logit_p = (
        intercept
        + beta_temp * df_congo["Temperature_std"].values
        + beta_temp2 * df_congo["Temperature_std"].values**2
        + beta_precip * df_congo["Precipitation_std"].values
        + beta_precip2 * df_congo["Precipitation_std"].values**2
    )
    p = pm.Deterministic("p", pm.math.sigmoid(logit_p))
    y_obs = pm.Binomial("y_obs", n=df_congo["Ex"].values, p=p, observed=df_congo["Pf"].values)

    trace_climate_congo = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 5. Baseline Model ---
with pm.Model() as baseline_model_congo:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    p = pm.Deterministic("p", pm.math.sigmoid(intercept))
    y_obs = pm.Binomial("y_obs", n=df_congo["Ex"].values, p=p, observed=df_congo["Pf"].values)

    trace_baseline_congo = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 6. Model Comparison Plot ---
p_climate_congo = trace_climate_congo.posterior["p"].mean(dim=["chain", "draw"]).values
p_baseline_congo = trace_baseline_congo.posterior["p"].mean(dim=["chain", "draw"]).values

plt.figure(figsize=(12, 6))
plt.plot(p_climate_congo, label="Climate Model", color='skyblue')
plt.plot(p_baseline_congo, label="Baseline Model", linestyle='--', color='salmon')
plt.ylabel("Predicted Malaria Prevalence")
plt.xlabel("Observation Index")
plt.legend()
plt.title("Climate vs Baseline Model: Congo/DRC")
plt.grid(True, linestyle=':')
plt.show()

# --- 7. Predicted vs Temperature (fixed Precipitation) ---
intercept_vals = trace_climate_congo.posterior["intercept"].stack(sample=("chain", "draw")).values
beta_temp_vals = trace_climate_congo.posterior["beta_temp"].stack(sample=("chain", "draw")).values
beta_precip_vals = trace_climate_congo.posterior["beta_precip"].stack(sample=("chain", "draw")).values
beta_temp2_vals = trace_climate_congo.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
beta_precip2_vals = trace_climate_congo.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

temp_range_raw = np.linspace(df_congo["Temperature"].min()-5, df_congo["Temperature"].max()+5, 100)
precip_const_raw = df_congo["Precipitation"].mean()
temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

pred_probs_temp = []
for temp_std_val in temp_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_std_val
        + beta_temp2_vals * temp_std_val**2
        + beta_precip_vals * precip_const_std
        + beta_precip2_vals * precip_const_std**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_temp.append(prob)

pred_probs_temp = np.array(pred_probs_temp)
mean_pred_temp = pred_probs_temp.mean(axis=1)
lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Temperature (Precip = {precip_const_raw:.2f} mm/day)")
plt.grid(True, linestyle=':')
plt.show()

# --- 8. Predicted vs Precipitation (fixed Temperature) ---
precip_range_raw = np.linspace(df_congo["Precipitation"].min()-5, df_congo["Precipitation"].max()+5, 100)
temp_const_raw = df_congo["Temperature"].mean()
precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

pred_probs_precip = []
for precip_std_val in precip_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_const_std
        + beta_temp2_vals * temp_const_std**2
        + beta_precip_vals * precip_std_val
        + beta_precip2_vals * precip_std_val**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_precip.append(prob)

pred_probs_precip = np.array(pred_probs_precip)
mean_pred_precip = pred_probs_precip.mean(axis=1)
lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Precipitation (Temp = {temp_const_raw:.2f}°C)")
plt.grid(True, linestyle=':')
plt.show()

# --- 9. Varying Precipitation (5 levels) ---
precip_levels = np.percentile(df_congo["Precipitation"], [10, 30, 50, 70, 90])
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

plt.figure(figsize=(10, 6))
for i, precip_val in enumerate(precip_levels):
    precip_std = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
    preds = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
            beta_precip_vals * precip_std + beta_precip2_vals * precip_std**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Temperature (5 Precip Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.show()

# --- 10. Varying Temperature (5 levels) ---
temp_levels = np.percentile(df_congo["Temperature"], [10, 30, 50, 70, 90])
colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

plt.figure(figsize=(10, 6))
for i, temp_val in enumerate(temp_levels):
    temp_std = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
    preds = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std + beta_temp2_vals * temp_std**2 +
            beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Precipitation (5 Temp Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.show()

# --- 11. Summary ---
print("\n--- Climate Model Summary ---")
print(az.summary(trace_climate_congo, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

print("\n--- Baseline Model Summary ---")
print(az.summary(trace_baseline_congo, var_names=["intercept"]))

# --- 12. Model Evaluation Metrics (added part) ---
# Calculate actual malaria prevalence
y_true = df_congo["PfPR2-10"]

# Predicted prevalence from the climate model
y_pred_climate = p_climate_congo

mae_climate = mean_absolute_error(y_true, y_pred_climate)
r2_climate = r2_score(y_true, y_pred_climate)
mse_climate = mean_squared_error(y_true, y_pred_climate)
rmse_climate = np.sqrt(mse_climate)

# Predicted prevalence from the baseline model (needs to be broadcast to the same shape as y_true)
y_pred_baseline_scalar = p_baseline_congo
y_pred_baseline = np.full_like(y_true, y_pred_baseline_scalar)

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)
mse_baseline = mean_squared_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)

print("\n--- Model Performance Metrics ---")
print("\nClimate Model:")
print(f"Mean Absolute Error (MAE): {mae_climate:.4f}")
print(f"R-squared (R2): {r2_climate:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_climate:.4f}")

print("\nBaseline Model:")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")
print(f"R-squared (R2): {r2_baseline:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_baseline:.4f}")

# Mozambique

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    raise

# Filter for Mozambique
df_mozambique = df[df["COUNTRY"] == "Mozambique"].copy()

try:
    df_temp = pd.read_csv("MozambiqueTemp_1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("MozambiquePrecipitation_1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files for Mozambique not found. Please ensure they are in the correct directory.")
    raise

climate_min_year = 1984
climate_max_year = 2020
df_mozambique = df_mozambique[(df_mozambique['YY'] >= climate_min_year) & (df_mozambique['YY'] <= climate_max_year)].copy()

print(f"Shape of df_mozambique after initial country and YEAR filter: {df_mozambique.shape}")
print(f"Years: Min={df_mozambique['YY'].min() if not df_mozambique.empty else 'N/A'}, Max={df_mozambique['YY'].max() if not df_mozambique.empty else 'N/A'}")

if df_mozambique.empty:
    print("\nError: df_mozambique is empty after filtering. Cannot proceed with analysis.")
    raise

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

df_mozambique["Lat_Rounded"] = df_mozambique["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_mozambique["Long_Rounded"] = df_mozambique["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_mozambique["Month_Name"] = df_mozambique["MM"].map(month_map)
df_mozambique["Temperature"] = np.nan
df_mozambique["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_mozambique.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_mozambique.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_mozambique.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation ---
df_mozambique = df_mozambique.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_mozambique = df_mozambique[(df_mozambique["Ex"] > 0) & (df_mozambique["Pf"] <= df_mozambique["Ex"])]
df_mozambique["Pf"] = df_mozambique["Pf"].astype(int)
df_mozambique["Ex"] = df_mozambique["Ex"].astype(int)

if df_mozambique.empty:
    print("\nError: No valid data points remaining after cleaning. Cannot proceed with modeling.")
    raise

temp_mean = df_mozambique["Temperature"].mean()
temp_std = df_mozambique["Temperature"].std()
precip_mean = df_mozambique["Precipitation"].mean()
precip_std = df_mozambique["Precipitation"].std()

df_mozambique["Temperature_std"] = (df_mozambique["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_mozambique["Precipitation_std"] = (df_mozambique["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
with pm.Model() as climate_model_mozambique:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
    beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
    beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
    beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

    logit_p = (
        intercept
        + beta_temp * df_mozambique["Temperature_std"].values
        + beta_temp2 * df_mozambique["Temperature_std"].values**2
        + beta_precip * df_mozambique["Precipitation_std"].values
        + beta_precip2 * df_mozambique["Precipitation_std"].values**2
    )
    p = pm.Deterministic("p", pm.math.sigmoid(logit_p))
    y_obs = pm.Binomial("y_obs", n=df_mozambique["Ex"].values, p=p, observed=df_mozambique["Pf"].values)

    trace_climate_mozambique = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 5. Baseline Model ---
with pm.Model() as baseline_model_mozambique:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    p = pm.Deterministic("p", pm.math.sigmoid(intercept))
    y_obs = pm.Binomial("y_obs", n=df_mozambique["Ex"].values, p=p, observed=df_mozambique["Pf"].values)

    trace_baseline_mozambique = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 6. Model Comparison Plot ---
p_climate_mozambique = trace_climate_mozambique.posterior["p"].mean(dim=["chain", "draw"]).values
p_baseline_mozambique = trace_baseline_mozambique.posterior["p"].mean(dim=["chain", "draw"]).values

plt.figure(figsize=(12, 6))
plt.plot(p_climate_mozambique, label="Climate Model", color='skyblue')
plt.plot(p_baseline_mozambique, label="Baseline Model", linestyle='--', color='salmon')
plt.ylabel("Predicted Malaria Prevalence")
plt.xlabel("Observation Index")
plt.legend()
plt.title("Climate vs Baseline Model: Mozambique")
plt.grid(True, linestyle=':')
plt.savefig("mozambique_prevalence_comparison.png")

# --- 7. Predicted vs Temperature (fixed Precipitation) ---
intercept_vals = trace_climate_mozambique.posterior["intercept"].stack(sample=("chain", "draw")).values
beta_temp_vals = trace_climate_mozambique.posterior["beta_temp"].stack(sample=("chain", "draw")).values
beta_precip_vals = trace_climate_mozambique.posterior["beta_precip"].stack(sample=("chain", "draw")).values
beta_temp2_vals = trace_climate_mozambique.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
beta_precip2_vals = trace_climate_mozambique.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

temp_range_raw = np.linspace(df_mozambique["Temperature"].min()-5, df_mozambique["Temperature"].max()+5, 100)
precip_const_raw = df_mozambique["Precipitation"].mean()
temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

pred_probs_temp = []
for temp_std_val in temp_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_std_val
        + beta_temp2_vals * temp_std_val**2
        + beta_precip_vals * precip_const_std
        + beta_precip2_vals * precip_const_std**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_temp.append(prob)

pred_probs_temp = np.array(pred_probs_temp)
mean_pred_temp = pred_probs_temp.mean(axis=1)
lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Temperature for Mozambique (Precip = {precip_const_raw:.2f} mm/day)")
plt.grid(True, linestyle=':')
plt.savefig("mozambique_prevalence_vs_temperature.png")

# --- 8. Predicted vs Precipitation (fixed Temperature) ---
precip_range_raw = np.linspace(df_mozambique["Precipitation"].min()-5, df_mozambique["Precipitation"].max()+5, 100)
temp_const_raw = df_mozambique["Temperature"].mean()
precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

pred_probs_precip = []
for precip_std_val in precip_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_const_std
        + beta_temp2_vals * temp_const_std**2
        + beta_precip_vals * precip_std_val
        + beta_precip2_vals * precip_std_val**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_precip.append(prob)

pred_probs_precip = np.array(pred_probs_precip)
mean_pred_precip = pred_probs_precip.mean(axis=1)
lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Precipitation for Mozambique (Temp = {temp_const_raw:.2f}°C)")
plt.grid(True, linestyle=':')
plt.savefig("mozambique_prevalence_vs_precipitation.png")

# --- 9. Varying Precipitation (5 levels) ---
precip_levels = np.percentile(df_mozambique["Precipitation"], [10, 30, 50, 70, 90])
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

plt.figure(figsize=(10, 6))
for i, precip_val in enumerate(precip_levels):
    precip_std_val_fixed = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
    preds = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
            beta_precip_vals * precip_std_val_fixed + beta_precip2_vals * precip_std_val_fixed**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Temperature for Mozambique (5 Precip Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.savefig("mozambique_prevalence_vs_temperature_5precip.png")

# --- 10. Varying Temperature (5 levels) ---
temp_levels = np.percentile(df_mozambique["Temperature"], [10, 30, 50, 70, 90])
colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

plt.figure(figsize=(10, 6))
for i, temp_val in enumerate(temp_levels):
    temp_std_val_fixed = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
    preds = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val_fixed + beta_temp2_vals * temp_std_val_fixed**2 +
            beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Precipitation for Mozambique (5 Temp Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.savefig("mozambique_prevalence_vs_precipitation_5temp.png")

# --- 11. Summary ---
print("\n--- Climate Model Summary for Mozambique ---")
print(az.summary(trace_climate_mozambique, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

print("\n--- Baseline Model Summary for Mozambique ---")
print(az.summary(trace_baseline_mozambique, var_names=["intercept"]))

In [ ]:
# --- Scatter Plots ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(df_mozambique["Temperature"], df_mozambique["Pf"] / df_mozambique["Ex"] * 100, alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Mozambique: Temperature vs Malaria Prevalence")

plt.subplot(1, 2, 2)
plt.scatter(df_mozambique["Precipitation"], df_mozambique["Pf"] / df_mozambique["Ex"] * 100, alpha=0.6, color='green')
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Mozambique: Precipitation vs Malaria Prevalence")

plt.tight_layout()
plt.show()

# --- Monthly Climate and Prevalence Trends ---
# First compute PfPR2-10 from Pf and Ex
df_mozambique["PfPR2-10"] = df_mozambique["Pf"] / df_mozambique["Ex"] * 100

monthly_temp_mz = df_mozambique.groupby("MM")["Temperature"].mean()
monthly_precip_mz = df_mozambique.groupby("MM")["Precipitation"].mean()
monthly_pfpr_mz = df_mozambique.groupby("MM")["PfPR2-10"].mean()

plt.figure(figsize=(15, 6))

plt.subplot(2, 3, 1)
monthly_temp_mz.plot(marker='o')
plt.title("Mozambique: Seasonal Temperature Pattern")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")

plt.subplot(2, 3, 2)
monthly_precip_mz.plot(marker='o', color='blue')
plt.title("Mozambique: Seasonal Precipitation Pattern")
plt.xlabel("Month")
plt.ylabel("Precipitation (mm/day)")

plt.subplot(2, 3, 3)
monthly_pfpr_mz.plot(marker='o', color='red')
plt.title("Mozambique: Seasonal Malaria Pattern")
plt.xlabel("Month")
plt.ylabel("PfPR2-10 (%)")

# --- Heatmap of Binned Temp × Precip vs Prevalence ---
df_mozambique["T_bin"] = pd.cut(df_mozambique["Temperature"], bins=5, labels=False)
df_mozambique["P_bin"] = pd.cut(df_mozambique["Precipitation"], bins=5, labels=False)
heatmap_data_mz = df_mozambique.groupby(["T_bin", "P_bin"])["PfPR2-10"].mean().unstack()

plt.subplot(2, 3, 6)
sns.heatmap(heatmap_data_mz, cmap="YlOrRd", annot=True, fmt=".1f", cbar_kws={'label': 'PfPR2-10 (%)'})
plt.title("Mozambique: Climate-Malaria Heatmap")
plt.xlabel("Precipitation Bins")
plt.ylabel("Temperature Bins")

plt.tight_layout()
plt.show()


In [ ]:
# --- 12. Model Evaluation Metrics (added part) ---
# Calculate actual malaria prevalence
y_true = df_mozambique["PfPR2-10"]

# Predicted prevalence from the climate model
y_pred_climate = p_climate_mozambique

mae_climate = mean_absolute_error(y_true, y_pred_climate)
r2_climate = r2_score(y_true, y_pred_climate)
mse_climate = mean_squared_error(y_true, y_pred_climate)
rmse_climate = np.sqrt(mse_climate)

# Predicted prevalence from the baseline model (needs to be broadcast to the same shape as y_true)
y_pred_baseline_scalar = p_baseline_congo
y_pred_baseline = np.full_like(y_true, y_pred_baseline_scalar)

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)
mse_baseline = mean_squared_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)

print("\n--- Model Performance Metrics ---")
print("\nClimate Model:")
print(f"Mean Absolute Error (MAE): {mae_climate:.4f}")
print(f"R-squared (R2): {r2_climate:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_climate:.4f}")

print("\nBaseline Model:")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")
print(f"R-squared (R2): {r2_baseline:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_baseline:.4f}")

# Nigeria

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    raise

# Filter for Nigeria
df_nigeria = df[df["COUNTRY"] == "Nigeria"].copy()

try:
    df_temp = pd.read_csv("NigeriaTemp1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("NigeriaPrecipitation1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files for Nigeria not found. Please ensure they are in the correct directory.")
    raise

climate_min_year = 1984
climate_max_year = 2020
df_nigeria = df_nigeria[(df_nigeria['YY'] >= climate_min_year) & (df_nigeria['YY'] <= climate_max_year)].copy()

print(f"Shape of df_nigeria after initial country and YEAR filter: {df_nigeria.shape}")
print(f"Years: Min={df_nigeria['YY'].min() if not df_nigeria.empty else 'N/A'}, Max={df_nigeria['YY'].max() if not df_nigeria.empty else 'N/A'}")

if df_nigeria.empty:
    print("\nError: df_nigeria is empty after filtering. Cannot proceed with analysis.")
    raise

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

df_nigeria["Lat_Rounded"] = df_nigeria["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_nigeria["Long_Rounded"] = df_nigeria["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_nigeria["Month_Name"] = df_nigeria["MM"].map(month_map)
df_nigeria["Temperature"] = np.nan
df_nigeria["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_nigeria.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_nigeria.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_nigeria.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation ---
df_nigeria = df_nigeria.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_nigeria = df_nigeria[(df_nigeria["Ex"] > 0) & (df_nigeria["Pf"] <= df_nigeria["Ex"])]
df_nigeria["Pf"] = df_nigeria["Pf"].astype(int)
df_nigeria["Ex"] = df_nigeria["Ex"].astype(int)

if df_nigeria.empty:
    print("\nError: No valid data points remaining after cleaning. Cannot proceed with modeling.")
    raise

temp_mean = df_nigeria["Temperature"].mean()
temp_std = df_nigeria["Temperature"].std()
precip_mean = df_nigeria["Precipitation"].mean()
precip_std = df_nigeria["Precipitation"].std()

df_nigeria["Temperature_std"] = (df_nigeria["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_nigeria["Precipitation_std"] = (df_nigeria["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
with pm.Model() as climate_model_nigeria:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
    beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
    beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
    beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

    logit_p = (
        intercept
        + beta_temp * df_nigeria["Temperature_std"].values
        + beta_temp2 * df_nigeria["Temperature_std"].values**2
        + beta_precip * df_nigeria["Precipitation_std"].values
        + beta_precip2 * df_nigeria["Precipitation_std"].values**2
    )
    p = pm.Deterministic("p", pm.math.sigmoid(logit_p))
    y_obs = pm.Binomial("y_obs", n=df_nigeria["Ex"].values, p=p, observed=df_nigeria["Pf"].values)

    trace_climate_nigeria = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 5. Baseline Model ---
with pm.Model() as baseline_model_nigeria:
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    p = pm.Deterministic("p", pm.math.sigmoid(intercept))
    y_obs = pm.Binomial("y_obs", n=df_nigeria["Ex"].values, p=p, observed=df_nigeria["Pf"].values)

    trace_baseline_nigeria = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

# --- 6. Model Comparison Plot ---
p_climate_nigeria = trace_climate_nigeria.posterior["p"].mean(dim=["chain", "draw"]).values
p_baseline_nigeria = trace_baseline_nigeria.posterior["p"].mean(dim=["chain", "draw"]).values

plt.figure(figsize=(12, 6))
plt.plot(p_climate_nigeria, label="Climate Model", color='skyblue')
plt.plot(p_baseline_nigeria, label="Baseline Model", linestyle='--', color='salmon')
plt.ylabel("Predicted Malaria Prevalence")
plt.xlabel("Observation Index")
plt.legend()
plt.title("Climate vs Baseline Model: Nigeria")
plt.grid(True, linestyle=':')
plt.savefig("nigeria_prevalence_comparison.png")

# --- 7. Predicted vs Temperature (fixed Precipitation) ---
intercept_vals = trace_climate_nigeria.posterior["intercept"].stack(sample=("chain", "draw")).values
beta_temp_vals = trace_climate_nigeria.posterior["beta_temp"].stack(sample=("chain", "draw")).values
beta_precip_vals = trace_climate_nigeria.posterior["beta_precip"].stack(sample=("chain", "draw")).values
beta_temp2_vals = trace_climate_nigeria.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
beta_precip2_vals = trace_climate_nigeria.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

temp_range_raw = np.linspace(df_nigeria["Temperature"].min()-5, df_nigeria["Temperature"].max()+5, 100)
precip_const_raw = df_nigeria["Precipitation"].mean()
temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

pred_probs_temp = []
for temp_std_val in temp_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_std_val
        + beta_temp2_vals * temp_std_val**2
        + beta_precip_vals * precip_const_std
        + beta_precip2_vals * precip_const_std**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_temp.append(prob)

pred_probs_temp = np.array(pred_probs_temp)
mean_pred_temp = pred_probs_temp.mean(axis=1)
lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Temperature for Nigeria (Precip = {precip_const_raw:.2f} mm/day)")
plt.grid(True, linestyle=':')
plt.savefig("nigeria_prevalence_vs_temperature.png")

# --- 8. Predicted vs Precipitation (fixed Temperature) ---
precip_range_raw = np.linspace(df_nigeria["Precipitation"].min()-5, df_nigeria["Precipitation"].max()+5, 100)
temp_const_raw = df_nigeria["Temperature"].mean()
precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

pred_probs_precip = []
for precip_std_val in precip_range_std:
    logit = (
        intercept_vals
        + beta_temp_vals * temp_const_std
        + beta_temp2_vals * temp_const_std**2
        + beta_precip_vals * precip_std_val
        + beta_precip2_vals * precip_std_val**2
    )
    prob = 1 / (1 + np.exp(-logit))
    pred_probs_precip.append(prob)

pred_probs_precip = np.array(pred_probs_precip)
mean_pred_precip = pred_probs_precip.mean(axis=1)
lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title(f"Prevalence vs Precipitation for Nigeria (Temp = {temp_const_raw:.2f}°C)")
plt.grid(True, linestyle=':')
plt.savefig("nigeria_prevalence_vs_precipitation.png")

# --- 9. Varying Precipitation (5 levels) ---
precip_levels = np.percentile(df_nigeria["Precipitation"], [10, 30, 50, 70, 90])
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

plt.figure(figsize=(10, 6))
for i, precip_val in enumerate(precip_levels):
    precip_std_val_fixed = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
    preds = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
            beta_precip_vals * precip_std_val_fixed + beta_precip2_vals * precip_std_val_fixed**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

plt.xlabel("Temperature (°C)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Temperature for Nigeria (5 Precip Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.savefig("nigeria_prevalence_vs_temperature_5precip.png")

# --- 10. Varying Temperature (5 levels) ---
temp_levels = np.percentile(df_nigeria["Temperature"], [10, 30, 50, 70, 90])
colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

plt.figure(figsize=(10, 6))
for i, temp_val in enumerate(temp_levels):
    temp_std_val_fixed = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
    preds = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals + beta_temp_vals * temp_std_val_fixed + beta_temp2_vals * temp_std_val_fixed**2 +
            beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
        )
        preds.append(1 / (1 + np.exp(-logit)))
    mean_preds = np.mean(preds, axis=1)
    plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

plt.xlabel("Precipitation (mm/day)")
plt.ylabel("Predicted Malaria Prevalence")
plt.title("Prevalence vs Precipitation for Nigeria (5 Temp Levels)")
plt.legend()
plt.grid(True, linestyle=':')
plt.savefig("nigeria_prevalence_vs_precipitation_5temp.png")

# --- 11. Summary ---
print("\n--- Climate Model Summary for Nigeria ---")
print(az.summary(trace_climate_nigeria, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

print("\n--- Baseline Model Summary for Nigeria ---")
print(az.summary(trace_baseline_nigeria, var_names=["intercept"]))

In [ ]:
# --- Scatter Plots ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(df_nigeria["Temperature"], df_nigeria["Pf"] / df_nigeria["Ex"] * 100, alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Nigeria: Temperature vs Malaria Prevalence")

plt.subplot(1, 2, 2)
plt.scatter(df_nigeria["Precipitation"], df_nigeria["Pf"] / df_nigeria["Ex"] * 100, alpha=0.6, color='green')
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Nigeria: Precipitation vs Malaria Prevalence")

plt.tight_layout()
plt.show()

# --- Monthly Climate and Prevalence Trends ---
# First compute PfPR2-10 from Pf and Ex
df_nigeria["PfPR2-10"] = df_nigeria["Pf"] / df_nigeria["Ex"] * 100

monthly_temp_nigeria = df_nigeria.groupby("MM")["Temperature"].mean()
monthly_precip_nigeria = df_nigeria.groupby("MM")["Precipitation"].mean()
monthly_pfpr_nigeria = df_nigeria.groupby("MM")["PfPR2-10"].mean()

plt.figure(figsize=(15, 6))

plt.subplot(2, 3, 1)
monthly_temp_nigeria.plot(marker='o')
plt.title("Nigeria: Seasonal Temperature Pattern")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")

plt.subplot(2, 3, 2)
monthly_precip_nigeria.plot(marker='o', color='blue')
plt.title("Nigeria: Seasonal Precipitation Pattern")
plt.xlabel("Month")
plt.ylabel("Precipitation (mm/day)")

plt.subplot(2, 3, 3)
monthly_pfpr_nigeria.plot(marker='o', color='red')
plt.title("Nigeria: Seasonal Malaria Pattern")
plt.xlabel("Month")
plt.ylabel("PfPR2-10 (%)")

# --- Heatmap of Binned Temp × Precip vs Prevalence ---
df_nigeria["T_bin"] = pd.cut(df_nigeria["Temperature"], bins=5, labels=False)
df_nigeria["P_bin"] = pd.cut(df_nigeria["Precipitation"], bins=5, labels=False)
heatmap_data_nigeria = df_nigeria.groupby(["T_bin", "P_bin"])["PfPR2-10"].mean().unstack()

plt.subplot(2, 3, 6)
sns.heatmap(heatmap_data_nigeria, cmap="YlOrRd", annot=True, fmt=".1f", cbar_kws={'label': 'PfPR2-10 (%)'})
plt.title("Nigeria: Climate-Malaria Heatmap")
plt.xlabel("Precipitation Bins")
plt.ylabel("Temperature Bins")

plt.tight_layout()
plt.show()


In [ ]:
# --- 12. Model Evaluation Metrics (added part) ---
# Calculate actual malaria prevalence
y_true = df_nigeria["PfPR2-10"]

# Predicted prevalence from the climate model
y_pred_climate = p_climate_nigeria

mae_climate = mean_absolute_error(y_true, y_pred_climate)
r2_climate = r2_score(y_true, y_pred_climate)
mse_climate = mean_squared_error(y_true, y_pred_climate)
rmse_climate = np.sqrt(mse_climate)

# Predicted prevalence from the baseline model (needs to be broadcast to the same shape as y_true)
y_pred_baseline_scalar = p_baseline_congo
y_pred_baseline = np.full_like(y_true, y_pred_baseline_scalar)

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)
mse_baseline = mean_squared_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)

print("\n--- Model Performance Metrics ---")
print("\nClimate Model:")
print(f"Mean Absolute Error (MAE): {mae_climate:.4f}")
print(f"R-squared (R2): {r2_climate:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_climate:.4f}")

print("\nBaseline Model:")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")
print(f"R-squared (R2): {r2_baseline:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_baseline:.4f}")

# Uganda

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    exit()

df_uganda = df[df["COUNTRY"] == "Uganda"].copy()

try:
    df_temp = pd.read_csv("UgandaTemp_1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("UgandaPrecipitation_1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files for Uganda not found. Please ensure they are in the correct directory.")
    exit()

climate_min_year = 1984
climate_max_year = 2020
df_uganda = df_uganda[(df_uganda['YY'] >= climate_min_year) & (df_uganda['YY'] <= climate_max_year)].copy()

print(f"Shape of df_uganda after initial country and YEAR filter: {df_uganda.shape}")
print(f"Years: Min={df_uganda['YY'].min() if not df_uganda.empty else 'N/A'}, Max={df_uganda['YY'].max() if not df_uganda.empty else 'N/A'}")

if df_uganda.empty:
    print("\nError: df_uganda is empty after filtering. Cannot proceed with analysis.")
    print("Please check if 'Uganda' data exists in 'AfricaDataset.csv' for years 1984-2020.")
    exit()

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

# Apply rounding to match climate data grid resolution (0.5 Lat, 0.625 Lon)
df_uganda["Lat_Rounded"] = df_uganda["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_uganda["Long_Rounded"] = df_uganda["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_uganda["Month_Name"] = df_uganda["MM"].map(month_map)
df_uganda["Temperature"] = np.nan
df_uganda["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_uganda.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_uganda.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_uganda.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation
df_uganda = df_uganda.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_uganda = df_uganda[(df_uganda["Ex"] > 0) & (df_uganda["Pf"] <= df_uganda["Ex"])]
df_uganda["Pf"] = df_uganda["Pf"].astype(int)
df_uganda["Ex"] = df_uganda["Ex"].astype(int)

if df_uganda.empty:
    print("\nError: No valid data points remaining after cleaning and merging climate data for Uganda. Cannot proceed with modeling.")
    exit()

# Standardize climate variables
temp_mean = df_uganda["Temperature"].mean()
temp_std = df_uganda["Temperature"].std()
precip_mean = df_uganda["Precipitation"].mean()
precip_std = df_uganda["Precipitation"].std()

# Avoid division by zero if std dev is 0 (e.g., if all values are identical)
df_uganda["Temperature_std"] = (df_uganda["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_uganda["Precipitation_std"] = (df_uganda["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
try:
    with pm.Model() as climate_model_uganda:
        intercept = pm.Normal("intercept", mu=0, sigma=2)
        beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
        beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
        beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
        beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

        # Logit link function for binomial model
        logit_p = (
            intercept
            + beta_temp * df_uganda["Temperature_std"].values
            + beta_temp2 * df_uganda["Temperature_std"].values**2
            + beta_precip * df_uganda["Precipitation_std"].values
            + beta_precip2 * df_uganda["Precipitation_std"].values**2
        )
        p = pm.Deterministic("p", pm.math.sigmoid(logit_p)) # Convert log-odds to probability

        # Likelihood: Malaria cases (Pf) observed from total examined (Ex) following binomial distribution
        y_obs = pm.Binomial("y_obs", n=df_uganda["Ex"].values, p=p, observed=df_uganda["Pf"].values)

        # Sample from the posterior distribution
        trace_climate_uganda = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

    # --- 5. Baseline Model ---
    with pm.Model() as baseline_model_uganda:
        intercept = pm.Normal("intercept", mu=0, sigma=2)
        p = pm.Deterministic("p", pm.math.sigmoid(intercept))
        y_obs = pm.Binomial("y_obs", n=df_uganda["Ex"].values, p=p, observed=df_uganda["Pf"].values)

        trace_baseline_uganda = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

    # --- 6. Model Comparison Plot ---
    p_climate_uganda = trace_climate_uganda.posterior["p"].mean(dim=["chain", "draw"]).values
    p_baseline_uganda = trace_baseline_uganda.posterior["p"].mean(dim=["chain", "draw"]).values

    plt.figure(figsize=(12, 6))
    plt.plot(p_climate_uganda, label="Climate Model", color='skyblue')
    plt.plot(p_baseline_uganda, label="Baseline Model", linestyle='--', color='salmon')
    plt.ylabel("Predicted Malaria Prevalence")
    plt.xlabel("Observation Index")
    plt.legend()
    plt.title("Climate vs Baseline Model: Uganda")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()
    # --- 7. Predicted vs Temperature (fixed Precipitation) ---
    # Extract posterior samples for coefficients
    intercept_vals = trace_climate_uganda.posterior["intercept"].stack(sample=("chain", "draw")).values
    beta_temp_vals = trace_climate_uganda.posterior["beta_temp"].stack(sample=("chain", "draw")).values
    beta_precip_vals = trace_climate_uganda.posterior["beta_precip"].stack(sample=("chain", "draw")).values
    beta_temp2_vals = trace_climate_uganda.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
    beta_precip2_vals = trace_climate_uganda.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

    # Define a range for temperature and fix precipitation at its mean
    temp_range_raw = np.linspace(df_uganda["Temperature"].min() - 5, df_uganda["Temperature"].max() + 5, 100)
    precip_const_raw = df_uganda["Precipitation"].mean()

    # Standardize these values
    temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
    precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

    # Calculate predicted probabilities across the temperature range
    pred_probs_temp = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals
            + beta_temp_vals * temp_std_val
            + beta_temp2_vals * temp_std_val**2
            + beta_precip_vals * precip_const_std
            + beta_precip2_vals * precip_const_std**2
        )
        prob = 1 / (1 + np.exp(-logit))
        pred_probs_temp.append(prob)

    pred_probs_temp = np.array(pred_probs_temp)
    mean_pred_temp = pred_probs_temp.mean(axis=1)
    lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
    upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
    plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
    plt.xlabel("Temperature (°C)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title(f"Prevalence vs Temperature for Uganda (Precip = {precip_const_raw:.2f} mm/day)")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 8. Predicted vs Precipitation (fixed Temperature) ---
    # Define a range for precipitation and fix temperature at its mean
    precip_range_raw = np.linspace(df_uganda["Precipitation"].min() - 5, df_uganda["Precipitation"].max() + 5, 100)
    temp_const_raw = df_uganda["Temperature"].mean()

    # Standardize these values
    precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
    temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

    # Calculate predicted probabilities across the precipitation range
    pred_probs_precip = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals
            + beta_temp_vals * temp_const_std
            + beta_temp2_vals * temp_const_std**2
            + beta_precip_vals * precip_std_val
            + beta_precip2_vals * precip_std_val**2
        )
        prob = 1 / (1 + np.exp(-logit))
        pred_probs_precip.append(prob)

    pred_probs_precip = np.array(pred_probs_precip)
    mean_pred_precip = pred_probs_precip.mean(axis=1)
    lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
    upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
    plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
    plt.xlabel("Precipitation (mm/day)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title(f"Prevalence vs Precipitation for Uganda (Temp = {temp_const_raw:.2f}°C)")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 9. Varying Precipitation (5 levels) ---
    precip_levels = np.percentile(df_uganda["Precipitation"], [10, 30, 50, 70, 90])
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

    plt.figure(figsize=(10, 6))
    for i, precip_val in enumerate(precip_levels):
        precip_std_val_fixed = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
        preds = []
        for temp_std_val in temp_range_std:
            logit = (
                intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
                beta_precip_vals * precip_std_val_fixed + beta_precip2_vals * precip_std_val_fixed**2
            )
            preds.append(1 / (1 + np.exp(-logit)))
        mean_preds = np.mean(preds, axis=1)
        plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

    plt.xlabel("Temperature (°C)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title("Prevalence vs Temperature for Uganda (5 Precip Levels)")
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 10. Varying Temperature (5 levels) ---
    temp_levels = np.percentile(df_uganda["Temperature"], [10, 30, 50, 70, 90])
    colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

    plt.figure(figsize=(10, 6))
    for i, temp_val in enumerate(temp_levels):
        temp_std_val_fixed = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
        preds = []
        for precip_std_val in precip_range_std:
            logit = (
                intercept_vals + beta_temp_vals * temp_std_val_fixed + beta_temp2_vals * temp_std_val_fixed**2 +
                beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
            )
            preds.append(1 / (1 + np.exp(-logit)))
        mean_preds = np.mean(preds, axis=1)
        plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

    plt.xlabel("Precipitation (mm/day)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title("Prevalence vs Precipitation for Uganda (5 Temp Levels)")
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 11. Summary ---
    print("\n--- Climate Model Summary for Uganda ---")
    print(az.summary(trace_climate_uganda, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

    print("\n--- Baseline Model Summary for Uganda ---")
    print(az.summary(trace_baseline_uganda, var_names=["intercept"]))

except ImportError:
    print("\nError: PyMC library not found. Please install PyMC in your Python environment to run the Bayesian models.")
    print("You can usually install it using: pip install pymc")
except Exception as e:
    print(f"\nAn unexpected error occurred during the modeling or plotting steps: {e}")

In [ ]:
# --- Scatter Plots ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(df_uganda["Temperature"], df_uganda["Pf"] / df_uganda["Ex"] * 100, alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Uganda: Temperature vs Malaria Prevalence")

plt.subplot(1, 2, 2)
plt.scatter(df_uganda["Precipitation"], df_uganda["Pf"] / df_uganda["Ex"] * 100, alpha=0.6, color='green')
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Uganda: Precipitation vs Malaria Prevalence")

plt.tight_layout()
plt.show()

# --- Monthly Climate and Prevalence Trends ---
df_uganda["PfPR2-10"] = df_uganda["Pf"] / df_uganda["Ex"] * 100

monthly_temp_uganda = df_uganda.groupby("MM")["Temperature"].mean()
monthly_precip_uganda = df_uganda.groupby("MM")["Precipitation"].mean()
monthly_pfpr_uganda = df_uganda.groupby("MM")["PfPR2-10"].mean()

plt.figure(figsize=(15, 6))

plt.subplot(2, 3, 1)
monthly_temp_uganda.plot(marker='o')
plt.title("Uganda: Seasonal Temperature Pattern")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")

plt.subplot(2, 3, 2)
monthly_precip_uganda.plot(marker='o', color='blue')
plt.title("Uganda: Seasonal Precipitation Pattern")
plt.xlabel("Month")
plt.ylabel("Precipitation (mm/day)")

plt.subplot(2, 3, 3)
monthly_pfpr_uganda.plot(marker='o', color='red')
plt.title("Uganda: Seasonal Malaria Pattern")
plt.xlabel("Month")
plt.ylabel("PfPR2-10 (%)")

# --- Heatmap of Binned Temp × Precip vs Prevalence ---
df_uganda["T_bin"] = pd.cut(df_uganda["Temperature"], bins=5, labels=False)
df_uganda["P_bin"] = pd.cut(df_uganda["Precipitation"], bins=5, labels=False)
heatmap_data_uganda = df_uganda.groupby(["T_bin", "P_bin"])["PfPR2-10"].mean().unstack()

plt.subplot(2, 3, 6)
sns.heatmap(heatmap_data_uganda, cmap="YlOrRd", annot=True, fmt=".1f", cbar_kws={'label': 'PfPR2-10 (%)'})
plt.title("Uganda: Climate-Malaria Heatmap")
plt.xlabel("Precipitation Bins")
plt.ylabel("Temperature Bins")

plt.tight_layout()
plt.show()


In [ ]:
# --- 12. Model Evaluation Metrics (added part) ---
# Calculate actual malaria prevalence
y_true = df_uganda["PfPR2-10"]

# Predicted prevalence from the climate model
y_pred_climate = p_climate_uganda

mae_climate = mean_absolute_error(y_true, y_pred_climate)
r2_climate = r2_score(y_true, y_pred_climate)
mse_climate = mean_squared_error(y_true, y_pred_climate)
rmse_climate = np.sqrt(mse_climate)

# Predicted prevalence from the baseline model (needs to be broadcast to the same shape as y_true)
y_pred_baseline_scalar = p_baseline_congo
y_pred_baseline = np.full_like(y_true, y_pred_baseline_scalar)

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)
mse_baseline = mean_squared_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)

print("\n--- Model Performance Metrics ---")
print("\nClimate Model:")
print(f"Mean Absolute Error (MAE): {mae_climate:.4f}")
print(f"R-squared (R2): {r2_climate:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_climate:.4f}")

print("\nBaseline Model:")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")
print(f"R-squared (R2): {r2_baseline:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_baseline:.4f}")

# Tanzania

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

# --- 1. Data Loading and Filtering ---
try:
    df = pd.read_csv("/content/AfricaDataset.csv.csv")
except FileNotFoundError:
    print("Error: AfricaDataset.csv not found. Please ensure it's in the correct directory.")
    exit()

# Filter for Tanzania
df_tanzania = df[df["COUNTRY"] == "Tanzania"].copy()

try:
    df_temp = pd.read_csv("TanzaniaTemp_1984_2020.csv", skiprows=9)
    df_precip = pd.read_csv("TanzaniaPrecipitation_1984_2020.csv", skiprows=9)
except FileNotFoundError:
    print("Error: Climate data files for Tanzania not found. Please ensure they are in the correct directory.")
    exit()

climate_min_year = 1984
climate_max_year = 2020
df_tanzania = df_tanzania[(df_tanzania['YY'] >= climate_min_year) & (df_tanzania['YY'] <= climate_max_year)].copy()

print(f"Shape of df_tanzania after initial country and YEAR filter: {df_tanzania.shape}")
print(f"Years: Min={df_tanzania['YY'].min() if not df_tanzania.empty else 'N/A'}, Max={df_tanzania['YY'].max() if not df_tanzania.empty else 'N/A'}")

if df_tanzania.empty:
    print("\nError: df_tanzania is empty after filtering. Cannot proceed with analysis.")
    print("Please check if 'Tanzania' data exists in 'AfricaDataset.csv' for years 1984-2020.")
    exit()

# --- 2. Climate Data Merging ---
df_temp.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)
df_precip.rename(columns={"YEAR": "YY", "LAT": "Lat_Climate", "LON": "Long_Climate"}, inplace=True)

def round_to_nearest_grid(coord, grid_size):
    return round(coord / grid_size) * grid_size

# Apply rounding to match climate data grid resolution (0.5 Lat, 0.625 Lon)
df_tanzania["Lat_Rounded"] = df_tanzania["Lat"].apply(lambda x: round_to_nearest_grid(x, 0.5))
df_tanzania["Long_Rounded"] = df_tanzania["Long"].apply(lambda x: round_to_nearest_grid(x, 0.625))

month_map = {
    1: "JAN", 2: "FEB", 3: "MAR", 4: "APR", 5: "MAY", 6: "JUN",
    7: "JUL", 8: "AUG", 9: "SEP", 10: "OCT", 11: "NOV", 12: "DEC"
}
df_tanzania["Month_Name"] = df_tanzania["MM"].map(month_map)
df_tanzania["Temperature"] = np.nan
df_tanzania["Precipitation"] = np.nan

successful_temp_merges = 0
successful_precip_merges = 0

for index, row in df_tanzania.iterrows():
    year = row["YY"]
    lat_rounded = row["Lat_Rounded"]
    long_rounded = row["Long_Rounded"]
    month_name = row["Month_Name"]

    temp_row = df_temp[(df_temp["YY"] == year) &
                       (df_temp["Lat_Climate"] == lat_rounded) &
                       (df_temp["Long_Climate"] == long_rounded)]
    if not temp_row.empty and month_name in temp_row.columns:
        df_tanzania.loc[index, "Temperature"] = temp_row[month_name].values[0]
        successful_temp_merges += 1

    precip_row = df_precip[(df_precip["YY"] == year) &
                           (df_precip["Lat_Climate"] == lat_rounded) &
                           (df_precip["Long_Climate"] == long_rounded)]
    if not precip_row.empty and month_name in precip_row.columns:
        df_tanzania.loc[index, "Precipitation"] = precip_row[month_name].values[0]
        successful_precip_merges += 1

print(f"\nSuccessful Temperature Merges: {successful_temp_merges}")
print(f"Successful Precipitation Merges: {successful_precip_merges}")

# --- 3. Cleaning and Preparation
df_tanzania = df_tanzania.dropna(subset=["Pf", "Ex", "Temperature", "Precipitation"])
df_tanzania = df_tanzania[(df_tanzania["Ex"] > 0) & (df_tanzania["Pf"] <= df_tanzania["Ex"])]
df_tanzania["Pf"] = df_tanzania["Pf"].astype(int)
df_tanzania["Ex"] = df_tanzania["Ex"].astype(int)

if df_tanzania.empty:
    print("\nError: No valid data points remaining after cleaning and merging climate data for Tanzania. Cannot proceed with modeling.")
    exit()

# Standardize climate variables
temp_mean = df_tanzania["Temperature"].mean()
temp_std = df_tanzania["Temperature"].std()
precip_mean = df_tanzania["Precipitation"].mean()
precip_std = df_tanzania["Precipitation"].std()

# Avoid division by zero if std dev is 0 (e.g., if all values are identical)
df_tanzania["Temperature_std"] = (df_tanzania["Temperature"] - temp_mean) / (temp_std if temp_std != 0 else 1)
df_tanzania["Precipitation_std"] = (df_tanzania["Precipitation"] - precip_mean) / (precip_std if precip_std != 0 else 1)

# --- 4. Bayesian Climate Model ---
try:
    with pm.Model() as climate_model_tanzania:
        intercept = pm.Normal("intercept", mu=0, sigma=2)
        beta_temp = pm.Normal("beta_temp", mu=0, sigma=1)
        beta_precip = pm.Normal("beta_precip", mu=0, sigma=1)
        beta_temp2 = pm.Normal("beta_temp2", mu=-0.5, sigma=0.5)
        beta_precip2 = pm.Normal("beta_precip2", mu=0, sigma=1)

        # Logit link function for binomial model
        logit_p = (
            intercept
            + beta_temp * df_tanzania["Temperature_std"].values
            + beta_temp2 * df_tanzania["Temperature_std"].values**2
            + beta_precip * df_tanzania["Precipitation_std"].values
            + beta_precip2 * df_tanzania["Precipitation_std"].values**2
        )
        p = pm.Deterministic("p", pm.math.sigmoid(logit_p)) # Convert log-odds to probability

        # Likelihood: Malaria cases (Pf) observed from total examined (Ex) following binomial distribution
        y_obs = pm.Binomial("y_obs", n=df_tanzania["Ex"].values, p=p, observed=df_tanzania["Pf"].values)

        # Sample from the posterior distribution
        trace_climate_tanzania = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

    # --- 5. Baseline Model ---
    with pm.Model() as baseline_model_tanzania:
        intercept = pm.Normal("intercept", mu=0, sigma=2)
        p = pm.Deterministic("p", pm.math.sigmoid(intercept))
        y_obs = pm.Binomial("y_obs", n=df_tanzania["Ex"].values, p=p, observed=df_tanzania["Pf"].values)

        trace_baseline_tanzania = pm.sample(draws=2000, tune=2000, target_accept=0.9, random_seed=42, return_inferencedata=True)

    # --- 6. Model Comparison Plot ---
    p_climate_tanzania = trace_climate_tanzania.posterior["p"].mean(dim=["chain", "draw"]).values
    p_baseline_tanzania = trace_baseline_tanzania.posterior["p"].mean(dim=["chain", "draw"]).values

    plt.figure(figsize=(12, 6))
    plt.plot(p_climate_tanzania, label="Climate Model", color='skyblue')
    plt.plot(p_baseline_tanzania, label="Baseline Model", linestyle='--', color='salmon')
    plt.ylabel("Predicted Malaria Prevalence")
    plt.xlabel("Observation Index")
    plt.legend()
    plt.title("Climate vs Baseline Model: Tanzania")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()
    # --- 7. Predicted vs Temperature (fixed Precipitation) ---
    # Extract posterior samples for coefficients
    intercept_vals = trace_climate_tanzania.posterior["intercept"].stack(sample=("chain", "draw")).values
    beta_temp_vals = trace_climate_tanzania.posterior["beta_temp"].stack(sample=("chain", "draw")).values
    beta_precip_vals = trace_climate_tanzania.posterior["beta_precip"].stack(sample=("chain", "draw")).values
    beta_temp2_vals = trace_climate_tanzania.posterior["beta_temp2"].stack(sample=("chain", "draw")).values
    beta_precip2_vals = trace_climate_tanzania.posterior["beta_precip2"].stack(sample=("chain", "draw")).values

    # Define a range for temperature and fix precipitation at its mean
    temp_range_raw = np.linspace(df_tanzania["Temperature"].min() - 5, df_tanzania["Temperature"].max() + 5, 100)
    precip_const_raw = df_tanzania["Precipitation"].mean()

    # Standardize these values
    temp_range_std = (temp_range_raw - temp_mean) / (temp_std if temp_std != 0 else 1)
    precip_const_std = (precip_const_raw - precip_mean) / (precip_std if precip_std != 0 else 1)

    # Calculate predicted probabilities across the temperature range
    pred_probs_temp = []
    for temp_std_val in temp_range_std:
        logit = (
            intercept_vals
            + beta_temp_vals * temp_std_val
            + beta_temp2_vals * temp_std_val**2
            + beta_precip_vals * precip_const_std
            + beta_precip2_vals * precip_const_std**2
        )
        prob = 1 / (1 + np.exp(-logit))
        pred_probs_temp.append(prob)

    pred_probs_temp = np.array(pred_probs_temp)
    mean_pred_temp = pred_probs_temp.mean(axis=1)
    lower_temp = np.percentile(pred_probs_temp, 2.5, axis=1)
    upper_temp = np.percentile(pred_probs_temp, 97.5, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(temp_range_raw, mean_pred_temp, color='darkblue')
    plt.fill_between(temp_range_raw, lower_temp, upper_temp, color='skyblue', alpha=0.4)
    plt.xlabel("Temperature (°C)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title(f"Prevalence vs Temperature for Tanzania (Precip = {precip_const_raw:.2f} mm/day)")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 8. Predicted vs Precipitation (fixed Temperature) ---
    # Define a range for precipitation and fix temperature at its mean
    precip_range_raw = np.linspace(df_tanzania["Precipitation"].min() - 5, df_tanzania["Precipitation"].max() + 5, 100)
    temp_const_raw = df_tanzania["Temperature"].mean()

    # Standardize these values
    precip_range_std = (precip_range_raw - precip_mean) / (precip_std if precip_std != 0 else 1)
    temp_const_std = (temp_const_raw - temp_mean) / (temp_std if temp_std != 0 else 1)

    # Calculate predicted probabilities across the precipitation range
    pred_probs_precip = []
    for precip_std_val in precip_range_std:
        logit = (
            intercept_vals
            + beta_temp_vals * temp_const_std
            + beta_temp2_vals * temp_const_std**2
            + beta_precip_vals * precip_std_val
            + beta_precip2_vals * precip_std_val**2
        )
        prob = 1 / (1 + np.exp(-logit))
        pred_probs_precip.append(prob)

    pred_probs_precip = np.array(pred_probs_precip)
    mean_pred_precip = pred_probs_precip.mean(axis=1)
    lower_precip = np.percentile(pred_probs_precip, 2.5, axis=1)
    upper_precip = np.percentile(pred_probs_precip, 97.5, axis=1)

    plt.figure(figsize=(10, 6))
    plt.plot(precip_range_raw, mean_pred_precip, color='darkgreen')
    plt.fill_between(precip_range_raw, lower_precip, upper_precip, color='lightgreen', alpha=0.4)
    plt.xlabel("Precipitation (mm/day)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title(f"Prevalence vs Precipitation for Tanzania (Temp = {temp_const_raw:.2f}°C)")
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 9. Varying Precipitation (5 levels) ---
    precip_levels = np.percentile(df_tanzania["Precipitation"], [10, 30, 50, 70, 90])
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(precip_levels)))

    plt.figure(figsize=(10, 6))
    for i, precip_val in enumerate(precip_levels):
        precip_std_val_fixed = (precip_val - precip_mean) / (precip_std if precip_std != 0 else 1)
        preds = []
        for temp_std_val in temp_range_std:
            logit = (
                intercept_vals + beta_temp_vals * temp_std_val + beta_temp2_vals * temp_std_val**2 +
                beta_precip_vals * precip_std_val_fixed + beta_precip2_vals * precip_std_val_fixed**2
            )
            preds.append(1 / (1 + np.exp(-logit)))
        mean_preds = np.mean(preds, axis=1)
        plt.plot(temp_range_raw, mean_preds, label=f"Precip: {precip_val:.1f}", color=colors[i])

    plt.xlabel("Temperature (°C)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title("Prevalence vs Temperature for Tanzania (5 Precip Levels)")
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 10. Varying Temperature (5 levels) ---
    temp_levels = np.percentile(df_tanzania["Temperature"], [10, 30, 50, 70, 90])
    colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(temp_levels)))

    plt.figure(figsize=(10, 6))
    for i, temp_val in enumerate(temp_levels):
        temp_std_val_fixed = (temp_val - temp_mean) / (temp_std if temp_std != 0 else 1)
        preds = []
        for precip_std_val in precip_range_std:
            logit = (
                intercept_vals + beta_temp_vals * temp_std_val_fixed + beta_temp2_vals * temp_std_val_fixed**2 +
                beta_precip_vals * precip_std_val + beta_precip2_vals * precip_std_val**2
            )
            preds.append(1 / (1 + np.exp(-logit)))
        mean_preds = np.mean(preds, axis=1)
        plt.plot(precip_range_raw, mean_preds, label=f"Temp: {temp_val:.1f}", color=colors[i])

    plt.xlabel("Precipitation (mm/day)")
    plt.ylabel("Predicted Malaria Prevalence")
    plt.title("Prevalence vs Precipitation for Tanzania (5 Temp Levels)")
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.show()
    plt.close()

    # --- 11. Summary ---
    print("\n--- Climate Model Summary for Tanzania ---")
    print(az.summary(trace_climate_tanzania, var_names=["intercept", "beta_temp", "beta_precip", "beta_temp2", "beta_precip2"]))

    print("\n--- Baseline Model Summary for Tanzania ---")
    print(az.summary(trace_baseline_tanzania, var_names=["intercept"]))

except ImportError:
    print("\nError: PyMC library not found. Please install PyMC in your Python environment to run the Bayesian models.")
    print("You can usually install it using: pip install pymc")
except Exception as e:
    print(f"\nAn unexpected error occurred during the modeling or plotting steps: {e}")


In [ ]:
# --- Scatter Plots ---
import seaborn as sns
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(df_tanzania["Temperature"], df_tanzania["Pf"] / df_tanzania["Ex"] * 100, alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Tanzania: Temperature vs Malaria Prevalence")

plt.subplot(1, 2, 2)
plt.scatter(df_tanzania["Precipitation"], df_tanzania["Pf"] / df_tanzania["Ex"] * 100, alpha=0.6, color='green')
plt.xlabel("Precipitation (mm/day)")
plt.ylabel("PfPR2-10 (%)")
plt.title("Tanzania: Precipitation vs Malaria Prevalence")

plt.tight_layout()
plt.show()

# --- Monthly Climate and Prevalence Trends ---
# First compute PfPR2-10 from Pf and Ex
df_tanzania["PfPR2-10"] = df_tanzania["Pf"] / df_tanzania["Ex"] * 100

monthly_temp_tanzania = df_tanzania.groupby("MM")["Temperature"].mean()
monthly_precip_tanzania = df_tanzania.groupby("MM")["Precipitation"].mean()
monthly_pfpr_tanzania = df_tanzania.groupby("MM")["PfPR2-10"].mean()

plt.figure(figsize=(15, 6))

plt.subplot(2, 3, 1)
monthly_temp_tanzania.plot(marker='o')
plt.title("Tanzania: Seasonal Temperature Pattern")
plt.xlabel("Month")
plt.ylabel("Temperature (°C)")

plt.subplot(2, 3, 2)
monthly_precip_tanzania.plot(marker='o', color='blue')
plt.title("Tanzania: Seasonal Precipitation Pattern")
plt.xlabel("Month")
plt.ylabel("Precipitation (mm/day)")

plt.subplot(2, 3, 3)
monthly_pfpr_tanzania.plot(marker='o', color='red')
plt.title("Tanzania: Seasonal Malaria Pattern")
plt.xlabel("Month")
plt.ylabel("PfPR2-10 (%)")

# --- Heatmap of Binned Temp × Precip vs Prevalence ---
df_tanzania["T_bin"] = pd.cut(df_tanzania["Temperature"], bins=5, labels=False)
df_tanzania["P_bin"] = pd.cut(df_tanzania["Precipitation"], bins=5, labels=False)
heatmap_data_tanzania = df_tanzania.groupby(["T_bin", "P_bin"])["PfPR2-10"].mean().unstack()

plt.subplot(2, 3, 6)
sns.heatmap(heatmap_data_tanzania, cmap="YlOrRd", annot=True, fmt=".1f", cbar_kws={'label': 'PfPR2-10 (%)'})
plt.title("Tanzania: Climate-Malaria Heatmap")
plt.xlabel("Precipitation Bins")
plt.ylabel("Temperature Bins")

plt.tight_layout()
plt.show()


In [ ]:
# --- 12. Model Evaluation Metrics (added part) ---
# Calculate actual malaria prevalence
y_true = df_tanzania["PfPR2-10"]

# Predicted prevalence from the climate model
y_pred_climate = p_climate_tanzania

mae_climate = mean_absolute_error(y_true, y_pred_climate)
r2_climate = r2_score(y_true, y_pred_climate)
mse_climate = mean_squared_error(y_true, y_pred_climate)
rmse_climate = np.sqrt(mse_climate)

# Predicted prevalence from the baseline model (needs to be broadcast to the same shape as y_true)
y_pred_baseline_scalar = p_baseline_congo
y_pred_baseline = np.full_like(y_true, y_pred_baseline_scalar)

mae_baseline = mean_absolute_error(y_true, y_pred_baseline)
r2_baseline = r2_score(y_true, y_pred_baseline)
mse_baseline = mean_squared_error(y_true, y_pred_baseline)
rmse_baseline = np.sqrt(mse_baseline)

print("\n--- Model Performance Metrics ---")
print("\nClimate Model:")
print(f"Mean Absolute Error (MAE): {mae_climate:.4f}")
print(f"R-squared (R2): {r2_climate:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_climate:.4f}")

print("\nBaseline Model:")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")
print(f"R-squared (R2): {r2_baseline:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_baseline:.4f}")